# Model Output Notebook

<img style="float:center;" src="https://arcticexpansion.vse.gmu.edu/sites/arcticexpansion.vsnet.gmu.edu/files/images/header5d2.png" width=600px>

### ADCIRC-SWAN Output


### Initialize Libraries

In [1]:
import warnings;warnings.filterwarnings("ignore")
import netCDF4 as nc4;        import pandas as pd
import requests;              import json;
import matplotlib as mpl;     import matplotlib.pyplot as plt
import matplotlib.tri as tri; import pathlib as pl
import cartopy.crs as ccrs;   import cartopy.io.img_tiles as cart_img
import cartopy as cart;       import cartopy.io.img_tiles as cimgt
import numpy as np;           import xarray as xr
import skill_metrics as sm;   import geopandas as gpd
import multiprocessing as mp
from shapely import Polygon,Point,MultiPoint,LineString,MultiLineString;import shapely.vectorized
from sklearn.neighbors import BallTree
from scipy.stats import linregress

source $HOME/miniforge3/bin/activate


### Defined Functions

In [2]:
def noaa_data(begin,end,station,vdatum='NAVD',interval='6',
                       form='json',t_zone='GMT',unit='metric',product='water_level'):
    '''
      :param begin: begin date of the data
    :param end: end date of the data
    :param station: noaa station id
    :param vdatum: vertical datum such as NAVD
    :param interval: interval of the data (6 minutes)
    :param form: format of the data (json)
    :param t_zone: time zone of the data (GMT)
    :param unit: unit of the data (metric)
    :param product: type of the data (water_level)
    '''
    api = f'https://tidesandcurrents.noaa.gov/api/datagetter?begin_date={begin}&end_date={end}&station={station}'\
         f'&product={product}&application=NOS.COOPS.TAC.WL&datum={vdatum}&interval={interval}&time_zone={t_zone}&units={unit}&format={form}'
    data = requests.get(url=api).content.decode()
    return data


def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def point_lookup(model_lat:np.array, model_lon:np.array, satellite_lat:np.array, satellite_lon:np.array):
    tree = BallTree(np.deg2rad(np.c_[model_lat,model_lon]), metric='haversine')
    distances, indices = tree.query(np.deg2rad(np.c_[satellite_lat, satellite_lon]), k = 1)
    return distances*6371,indices


#### Data for this exercise can be found here
https://doi.org/10.17603/ds2-h0fw-2p96

Download the swan_HS.63.nc from one of the 4 folders

---

### Initialize path and read netcdf file

In [ ]:
root = pl.Path('/groups/ORC-CLIMATE/fhrl_repo/Arctic_Database/Raw_DATA')


In [ ]:

files = ['fort.63.nc','swan_HS.63.nc','swan_TPS.63.nc','swan_DIR.63.nc']
years = ['2006','2005','2004']


#['2024','2023','2022','2021','2020','2019',
        # '2018','2017','2016','2015','2014','2013',
        # '2012','2011','2010']
olat,olon = 70.148300, -143.704236
for year in years:
    data = {'time':[],'wse':[],'hs':[],'tp':[],'dir':[]}
    for file in files:
        ncfile = nc4.Dataset(root / year /'outputs' / file)
        start = pd.to_datetime(ncfile.variables['time'].base_date)
        dt = pd.date_range(start=start,freq='1h',periods=ncfile.dimensions['time'].size)
        x,y = ncfile.variables['x'][:],ncfile.variables['y'][:]
        distances,node_ids = wse.point_lookup(y, x,olat,olon)
        if len(data['time'])<1:
            data['time'] = dt[:]
        if file == 'fort.63.nc':
            model = ncfile.variables['zeta'][:,node_ids[0][0]]
            data['wse'] = model
        if file == 'swan_HS.63.nc':
            hs = ncfile.variables['swan_HS'][:,node_ids[0][0]]
            data['hs'] = hs
        if file == 'swan_TPS.63.nc':
            tp = ncfile.variables['swan_TPS'][:,node_ids[0][0]]
            data['tp'] = tp
        if file == 'swan_DIR.63.nc':
            dir = ncfile.variables['swan_DIR'][:,node_ids[0][0]]
            data['dir'] = dir
        ncfile.close()
    pdf = pd.DataFrame(data)
    pdf.to_csv('/scratch/tmiesse/project/data4tom/' + year + '_data.csv',index=False)